# Selecting SRM Targets with MSstats

[No more coefficients of variance](https://yaaminiv.github.io/Selecting-SRM-Targets-2/)! Let's use MSstats instead! This method is more statistically sound, and Emma has used a similar method to analyze DDA data for potential targets. I'm basing my workflow off of the [MSstats manual](https://bioconductor.org/packages/release/bioc/vignettes/MSstats/inst/doc/MSstats-manual.pdf).

## Step 1: Merge Normalized Peak Area with Protein GO Terms

The first thing I want to do is merge my normalized peak areas with [GO term information](https://github.com/RobertsLab/project-oyster-oa/blob/master/analyses/DNR_Skyline_20170512/Proteins-GO-terms.tabular). I did this in Galaxy.

![untitled](https://cloud.githubusercontent.com/assets/22335838/26030527/d6839bb6-380a-11e7-860e-1a36c261fda4.png)

![untitled](https://cloud.githubusercontent.com/assets/22335838/26030573/e3c72364-380b-11e7-98e8-d2ffc9437252.png)

## Step 2: Select Stress-Related Proteins

In [28]:
!grep "stress" 2017-05-12-normalized-protein-transitions-GO-terms.tabular | wc -l

    9959


In [29]:
!grep "stress" 2017-05-12-normalized-protein-transitions-GO-terms.tabular > 2017-05-12-normalized-protein-transitions-stress.txt

In [32]:
ls

2017-05-12-normalized-peak-areas.tabular
2017-05-12-normalized-protein-transitions-GO-terms.tabular
2017-05-12-normalized-protein-transitions-stress.txt
Proteins-GO-terms.tabular


I also deleted some columns with extraneous information. My spreadsheet looks like this:

![untitled](https://cloud.githubusercontent.com/assets/22335838/26030635/325ac62e-380d-11e7-9cf5-b19181a2a4c8.png)

## Step 3: Use MSstats to Find Differentially Expressed Peptides

MSstats is an R package that will allow me to figure out which peptides were differentially expressed between replicates. I'm going to use this to see what was differentially expressed between bare and eelgrass conditions. This is because Micah mentioned that oysters in eelgrass grew more than oysters outside of eelgrass.

### Step 3a: Add Condition and BioReplicate information

For this step, I'm adapting information from [Emma's lab notebook](https://www.evernote.com/shard/s242/sh/6beda768-71df-477c-8e50-5400c8bfc0d9/641c7a94006329064c9595052e47097f). Condition would refer to the combination of sample site and eelgrass presence. BioReplicate would either be 1 or 2, since I ran each sample in duplicate.

**Table 1**. Condition and BioReplicate information for MSstats.

| ID | DNR Vial | Sample Site | Eelgrass Presence | Replicate |
|:--:|:--------:|:-----------:|:-----------------:|:---------:|
|  1 |   O127   |      WB     |        Bare       |     1     |
|  2 |   O107   |      SK     |      Eelgrass     |     1     |
|  3 |    O07   |      CI     |      Eelgrass     |     1     |
|  4 |    O77   |      PG     |      Eelgrass     |     1     |
|  5 |    O47   |      FB     |        Bare       |     1     |
|  6 |    O55   |      PG     |        Bare       |     1     |
|  7 |    O37   |      FB     |      Eelgrass     |     1     |
|  8 |    O15   |      CI     |        Bare       |     1     |
|  9 |   O142   |      WB     |      Eelgrass     |     1     |
| 10 |   O119   |      SK     |        Bare       |     1     |
| 11 |    O47   |      FB     |        Bare       |     2     |
| 12 |   O127   |      WB     |        Bare       |     2     |
| 13 |    O37   |      FB     |      Eelgrass     |     2     |
| 14 |    O55   |      PG     |        Bare       |     2     |
| 15 |    O15   |      CI     |        Bare       |     2     |
| 16 |   O119   |      SK     |        Bare       |     2     |
| 18 |   O142   |      WB     |      Eelgrass     |     2     |
| 19 |    O07   |      CI     |      Eelgrass     |     2     |
| 20 |    O77   |      PG     |      Eelgrass     |     2     |
| 24 |   O107   |      SK     |      Eelgrass     |     2     |
| 25 |   O107   |      SK     |      Eelgrass     |     3     |

Under Settings >> Document Settings >> Annotation, I clicked "Edit List."

![image-2](https://cloud.githubusercontent.com/assets/22335838/26030812/9b636e04-3813-11e7-95f5-11284dfc6133.png)

I created clicked "Edit List" and then created a new Annotation for Conditions. I created a "Values List," and added the following conditions and clicked "OK":

- Bare
- Eelgrass


![image-3](https://cloud.githubusercontent.com/assets/22335838/26030810/9b629e34-3813-11e7-8cac-d3c01697a427.png)

I then added a "Number" list for BioReplicate and clicked "OK":

![image-4](https://cloud.githubusercontent.com/assets/22335838/26030811/9b62e588-3813-11e7-8ee0-09358cbe9189.png)

I then clicked "OK" under Define Annotations...

![image-5](https://cloud.githubusercontent.com/assets/22335838/26030816/9b785d3c-3813-11e7-9de4-6591549fee90.png)

...and checked the two boxes for "Condition" and "BioReplicate."

![image-6](https://cloud.githubusercontent.com/assets/22335838/26030815/9b775414-3813-11e7-9c68-031e2cdb7f01.png)

Under View >> Results Grid, I added the Condition and BioReplicate information to each sample based on Table 1.

![image-7](https://cloud.githubusercontent.com/assets/22335838/26030817/9b785cb0-3813-11e7-9839-a84775d3e2b5.png)
![image-8](https://cloud.githubusercontent.com/assets/22335838/26030814/9b764f6a-3813-11e7-9593-556869612a34.png)

### Step 3b: Export Report from Skyline

According to the Skyline manual, I need the following columns to use MSstats:

- ProteinName
- PeptideSequence
- PrecursorCharge
- FragmentIon
- ProductCharge
- IsotopeLabelType
- Condition
- BioReplicate
- Run (i.e. File Name)
- Intensity (i.e. Peak Area)

I exported the report with the following settings:

![unnamed-1](https://cloud.githubusercontent.com/assets/22335838/26030913/ba45de6c-3816-11e7-9e17-c765eb5a4944.png)
![unnamed-2](https://cloud.githubusercontent.com/assets/22335838/26030914/ba46ce12-3816-11e7-8aff-3583527081c2.png)
![unnamed-3](https://cloud.githubusercontent.com/assets/22335838/26030915/ba5a248a-3816-11e7-982e-a21dc2d2e637.png)

I saved the report at this [OWL link](http://owl.fish.washington.edu/spartina/DNR_Skyline_MSstats_20170513/2017-05-13-peptide-results-MSstats.csv).

In [ ]:
!curl http://owl.fish.washington.edu/spartina/DNR_Skyline_MSstats_20170513/2017-05-13-peptide-results-MSstats.csv \
> 2017-05-13-peptide-results-MSstats.csv

In [5]:
!head 2017-05-13-peptide-results-MSstats.csv

**UPDATE: At this point I stopped because I need to process my Skyline output in MSstats itself. I started a new notebook for these anl